In [127]:
import numpy as np
import pandas as pd
import re

In [128]:
GROUP_NAME = "💩Guerra dos Tronos💩"

In [129]:
with open('chat.txt', 'r', encoding='utf-8') as f:
    chat = f.read()

chat = re.sub('\u200e', '', chat)

re_message = r"(\[(\d+\/+\d+\/+\d+ \d+:+\d+:+\d+)\] ([^:]+):(.*))"
parsed_messages = re.findall(re_message, chat)

df_chat = pd.DataFrame(parsed_messages, 
                       columns=['Full Message', 'Date', 'Sender', 'Message'])

df_chat['Date'] = df_chat['Date'].apply(lambda x: pd.to_datetime(x, format="%d/%m/%Y %H:%M:%S"))
display(df_chat)

,Full Message,Date,Sender,Message
0,[04/07/2024 20:00:39] 💩Guerra dos Tronos💩: Les...,2024-07-04 20:00:39,💩Guerra dos Tronos💩,Les messages et les appels sont chiffrés de b...
1,[04/07/2024 20:00:39] Antonio Seixas: Antonio ...,2024-07-04 20:00:39,Antonio Seixas,Antonio Seixas a créé ce groupe
2,[04/07/2024 20:39:02] 💩Guerra dos Tronos💩: Ant...,2024-07-04 20:39:02,💩Guerra dos Tronos💩,Antonio Seixas vous a ajouté·e
3,[04/07/2024 20:39:12] Guilherme Piffer: Porra ...,2024-07-04 20:39:12,Guilherme Piffer,Porra na perrache é foda
4,[04/07/2024 20:39:17] Guilherme Piffer: Já fiz...,2024-07-04 20:39:17,Guilherme Piffer,Já fiz e não recomendo
...,...,...,...,...
1678,[18/07/2024 18:26:21] Leonardo Torres: 💩,2024-07-18 18:26:21,Leonardo Torres,💩
1679,[18/07/2024 18:27:45] Alice Martins: 💩,2024-07-18 18:27:45,Alice Martins,💩
1680,[18/07/2024 18:58:10] Henrique Lefundes: !,2024-07-18 18:58:10,Henrique Lefundes,!
1681,[18/07/2024 20:27:53] Daniel Piazzi: Daniel Pi...,2024-07-18 20:27:53,Daniel Piazzi,Daniel Piazzi a changé la description du groupe


In [130]:
# Select the time span to analyze
start_date = '2024-07-04 00:00:00'
end_date = '2024-07-18 23:59:59'

mask = (df_chat['Date'] > start_date) & (df_chat['Date'] <= end_date)
df_chat = df_chat.loc[mask]

In [131]:
def find_poop(message):
    matches = re.findall(u"\U0001F4A9", message)
    if matches:
        return True
    else: 
        return False
    
df_chat['Poop'] = df_chat['Message'].apply(find_poop)

In [132]:
def create_ranking(df):
    senders = df['Sender'].unique()

    ranking = []

    for sender in senders:
        n = df['Sender'].value_counts()[sender]
        ranking.append((sender, n))

    index = [element[0] for element in ranking]
    values = [element[1] for element in ranking]

    df_ranking = pd.DataFrame(values, columns=["Counter"], index=index)

    return df_ranking

df_chat_poop = df_chat[df_chat['Poop'] == True]
df_ranking = create_ranking(df_chat_poop)

In [133]:
# Adding the initial values
OFFSET_ANTONIO_SEIXAS = 0
counter = df_ranking.at['Antonio Seixas', 'Counter']
df_ranking.at['Antonio Seixas', 'Counter'] = counter + OFFSET_ANTONIO_SEIXAS

OFFSET_THIAGO_BITTENCOURT = 2
counter = df_ranking.at['Thiago Bittencourt', 'Counter']
df_ranking.at['Thiago Bittencourt', 'Counter'] = counter + OFFSET_THIAGO_BITTENCOURT

OFFSET_LEONARDO_TORRES = -1
counter = df_ranking.at['Leonardo Torres', 'Counter']
df_ranking.at['Leonardo Torres', 'Counter'] = counter + OFFSET_LEONARDO_TORRES

OFFSET_DANIEL_PIAZZI = 3
counter = df_ranking.at['Daniel Piazzi', 'Counter']
df_ranking.at['Daniel Piazzi', 'Counter'] = counter + OFFSET_DANIEL_PIAZZI

OFFSET_RAFAEL = 3
counter = df_ranking.at['Rafael', 'Counter']
df_ranking.at['Rafael', 'Counter'] = counter + OFFSET_RAFAEL

OFFSET_GUILHERME_PIFFER = 3
counter = df_ranking.at['Guilherme Piffer', 'Counter']
df_ranking.at['Guilherme Piffer', 'Counter'] = counter + OFFSET_GUILHERME_PIFFER

OFFSET_LAURA_DEBASTIANI = 2
counter = df_ranking.at['Laura Debastiani', 'Counter']
df_ranking.at['Laura Debastiani', 'Counter'] = counter + OFFSET_LAURA_DEBASTIANI

OFFSET_ALICE_MARTINS = 4
counter = df_ranking.at['Alice Martins', 'Counter']
df_ranking.at['Alice Martins', 'Counter'] = counter + OFFSET_ALICE_MARTINS

OFFSET_HENRIQUE_LEFUNDES = 3
counter = df_ranking.at['Henrique Lefundes', 'Counter']
df_ranking.at['Henrique Lefundes', 'Counter'] = counter + OFFSET_HENRIQUE_LEFUNDES

OFFSET_DUANE = 2
counter = df_ranking.at['Duane', 'Counter']
df_ranking.at['Duane', 'Counter'] = counter + OFFSET_DUANE

OFFSET_DUDA = 1
counter = df_ranking.at['Duda', 'Counter']
df_ranking.at['Duda', 'Counter'] = counter + OFFSET_DUDA

df_ranking = df_ranking.sort_values(by='Counter', ascending=False)
df_ranking

,Counter
Guilherme Piffer,33
Thiago Bittencourt,31
Antonio Seixas,26
Alice Martins,26
Leonardo Torres,25
Daniel Piazzi,25
Duda,23
Laura Debastiani,22
Henrique Lefundes,21
Rafael,19


In [134]:
def calculate_winner(df):
    max_value = df['Counter'].max()
    max_indexes = df[df['Counter'] == max_value].index

    return max_indexes

def calculate_most_poop_day(df):
    most_poops_day = df['Date'].dt.date.value_counts().idxmax()
    n = df['Date'].dt.date.value_counts().max()
    return most_poops_day, n

def calculate_most_poop_hour(df):
    most_poops_hour = df['Date'].dt.hour.value_counts().idxmax()
    n = df['Date'].dt.hour.value_counts().max()
    return most_poops_hour, n

def calculate_poops_period(df):
    morning = df[(df['Date'].dt.hour >= 6) & (df['Date'].dt.hour < 12)]
    afternoon = df[(df['Date'].dt.hour >= 12) & (df['Date'].dt.hour < 18)]
    night = df[(df['Date'].dt.hour >= 18) & (df['Date'].dt.hour < 24)]
    dawn = df[(df['Date'].dt.hour >= 0) & (df['Date'].dt.hour < 6)]

    return len(morning), len(afternoon), len(night), len(dawn)

def calculate_most_poop_period(df):
    morning, afternoon, night, dawn = calculate_poops_period(df)

    periods = {
        'Manhã': morning,
        'Tarde': afternoon,
        'Noite': night,
        'Madrugada': dawn
    }

    max_value = max(periods.values())
    max_period = [key for key, value in periods.items() if value == max_value]

    return max_period, max_value

In [135]:
def find_smallest_interval(sender_df):
    # Find the smallest interval between two poops in hours
    sender_df = sender_df.sort_values(by='Date')
    sender_df['Diff'] = sender_df['Date'].diff()
    sender_df['Diff'] = sender_df['Diff'].fillna(pd.Timedelta(seconds=0))
    sender_df['Diff'] = sender_df['Diff'].apply(lambda x: x.total_seconds())

    min_diff = sender_df['Diff'].sort_values().iloc[1]
    min_diff = pd.to_datetime(min_diff, unit='s').strftime("%d - %H:%M")

    days = int(min_diff.split(' - ')[0])
    hours = int(min_diff.split(' - ')[1].split(':')[0])
    minutes = int(min_diff.split(' - ')[1].split(':')[1])
    
    return days, hours, minutes

def find_largest_interval(sender_df):
    sender_df = sender_df.sort_values(by='Date')
    sender_df['Diff'] = sender_df['Date'].diff()
    sender_df['Diff'] = sender_df['Diff'].fillna(pd.Timedelta(seconds=0))
    sender_df['Diff'] = sender_df['Diff'].apply(lambda x: x.total_seconds())

    max_diff = sender_df['Diff'].sort_values().iloc[-1]
    max_diff = pd.to_datetime(max_diff, unit='s').strftime("%d - %H:%M")

    days = int(max_diff.split(' - ')[0])
    hours = int(max_diff.split(' - ')[1].split(':')[0])
    minutes = int(max_diff.split(' - ')[1].split(':')[1])
    
    return days, hours, minutes

def calculate_average(sender_df):
    # Calculate the average of poops per day
    days = sender_df['Date'].dt.date.unique()
    total_days = len(days)
    total_poops = sender_df.shape[0]

    return total_poops / total_days

def get_sender_df(df, sender):
    sender_df = df[df['Sender'] == sender]
    sender_df = sender_df[sender_df['Poop'] == True]
    return sender_df

In [136]:
def str_interval(days, hours, minutes):
    if days > 2:
        days_str = f"{days - 1} dias"
    elif days == 2:
        days_str = f"{days - 1} dia"
    else:
        days_str = ""

    if hours > 1:
        hours_str = f"{hours} horas"
    elif hours == 1:
        hours_str = f"{hours} hora"
    else:
        hours_str = ""

    if minutes > 1:
        minutes_str = f"{minutes} minutos"
    elif minutes == 1:
        minutes_str = f"{minutes} minuto"
    else:
        minutes_str = ""
    
    total_str = f"{days_str} {hours_str} {minutes_str}"
    total_str = total_str.strip()

    return total_str

def str_week_winners(winners, value):
    if len(winners) == 2:
        winners_str = ' e '.join(winners)
        winners_str = f"OS MAIORES CAGÕES DA SEMANA FORAM {winners_str} COM {value} CAGADAS"
    elif len(winners) > 2:
        winners_str = ', '.join(winners[:-1])
        winners_str = f"OS MAIORES CAGÕES DA SEMANA FORAM {winners_str} e {winners[-1]} COM {value} CAGADAS"
    else:
        winners_str = f"O MAIOR CAGÃO DA SEMANA FOI {winners[0]} COM {value} CAGADAS"
    
    return winners_str

def str_most_poop_day(day, n):
    return f"O dia com mais cagadas foi {day} com {n} cagadas"

def str_most_poop_hour(hour, n):
    return f"O horário mais ocupado da semana é às {hour} com {n} cagadas"

def str_periods(morning, afternoon, night, dawn):
    str_periods = f"""
                    🌅 Manhã (6h-12h): {morning} cagadas
                    🌄 Tarde (12h-8h): {afternoon} cagadas
                    🌃 Noite (18h-24h): {night} cagadas
                    🌉 Madrugada (0h-6h): {dawn} cagadas
                    """
    
    str_periods = [line.strip() for line in str_periods.split('\n') if line.strip() != '']
    return '\n'.join(str_periods)

def str_most_poop_period(period, n):
    return f"O período preferido do dia pra cagar é de {period} com {n} cagadas"

def str_day_winners(day, winners, value):
    day = day.strftime("%d/%m")
    if len(winners) == 2:
        winners_str = ' e '.join(winners)
        winners_str = f"📅🥇 OS MAIORES CAGÕES DO DIA {day} FORAM {winners_str} COM {value} CAGADAS 💩"
    elif len(winners) > 2:
        winners_str = ', '.join(winners[:-1])
        winners_str = f"📅🥇 OS MAIORES CAGÕES DO DIA {day} FORAM {winners_str} e {winners[-1]} COM {value} CAGADAS 💩"
    else:
        winners_str = f"📅🥇 O MAIOR CAGÃO DO DIA {day} FOI {winners[0]} COM {value} CAGADAS 💩"
    
    return winners_str

def print_general_resume(df, rank_df):
    start_date = df['Date'].min().strftime("%d/%m/%Y")
    end_date = df['Date'].max().strftime("%d/%m/%Y")

    df_chat_poop = df[df['Poop'] == True]

    winners = calculate_winner(rank_df)
    value = df_ranking.at[winners[0], 'Counter']

    most_poop_day, n = calculate_most_poop_day(df_chat_poop)
    most_poop_hour, n = calculate_most_poop_hour(df_chat_poop)
    morning, afternoon, night, dawn = calculate_poops_period(df_chat_poop)
    most_period, n = calculate_most_poop_period(df_chat_poop)

    str_resume = f"""
            💩🪠 RESUMO DA SEMANA {start_date} - {end_date} 🪠💩

            🥇 {str_week_winners(winners, value)} 🥇 

            📅 {str_most_poop_day(most_poop_day, n)}

            🕑 {str_most_poop_hour(most_poop_hour, n)}

            {str_periods(morning, afternoon, night, dawn)}
            {str_most_poop_period(most_period[0], n)}
          """
    
    str_resume = [line.strip() for line in str_resume.split('\n') if line.strip() != '']
    str_resume = '\n'.join(str_resume)

    print(str_resume)

def print_day_winners(df):
    days = df['Date'].dt.date.unique()
    df_chat_poop = df[df['Poop'] == True]

    for day in days:
        df_chat_poop_day = df_chat_poop[df_chat_poop['Date'].dt.date == day]

        df_ranking_day = create_ranking(df_chat_poop_day)

        values = df_ranking_day['Counter']

        max_value = values.max()
        max_indexes = values[values == max_value].index.tolist()

        print(str_day_winners(day, max_indexes, max_value), end='\n\n')

def print_ranking(df_ranking):
    len_ranking = len(df_ranking)
    print("🏆 RANKING SEMANAL 🏆")
    for i, (sender, value) in enumerate(df_ranking.iterrows()):
        if i == 0:
            print(f"🥇 {i+1}. {sender}: {value['Counter']} cagadas")
        elif i == 1:
            print(f"🥈 {i+1}. {sender}: {value['Counter']} cagadas")
        elif i == 2:
            print(f"🥉 {i+1}. {sender}: {value['Counter']} cagadas")
        elif i in range(len_ranking - 3, len_ranking):
            print(f"🫃 {i+1}. {sender}: {value['Counter']} cagadas (ACTIVIA ZONE)")
        else:
            print(f"🪠 {i+1}. {sender}: {value['Counter']} cagadas")

def print_personal_resume(df, rank_df, sender):
    sender_df = get_sender_df(df, sender)
    days_small, hours_small, minutes_small = find_smallest_interval(sender_df)
    days_large, hours_large, minutes_large = find_largest_interval(sender_df)
    average = calculate_average(sender_df)
    rank_position = rank_df.index.get_loc(sender) + 1

    str_resume = f"""
    🧻🪠 RESUMO PESSOAL @{sender} 🪠🧻
    ⏳ Maior intervalo sem cagar: {str_interval(days_large, hours_large, minutes_large)}
    ⌛ Menor intervalo sem cagar: {str_interval(days_small, hours_small, minutes_small)}
    🚽 Caga em média {average:.2f} vezes por dia
    🧻 Está na posição {rank_position} do ranking com {rank_df.at[sender, 'Counter']} cagadas"""

    str_resume = [line.strip() for line in str_resume.split('\n') if line.strip() != '']
    str_resume = '\n'.join(str_resume)

    print(str_resume, end='\n\n')

In [137]:
print_general_resume(df_chat, df_ranking)

💩🪠 RESUMO DA SEMANA 04/07/2024 - 18/07/2024 🪠💩
🥇 O MAIOR CAGÃO DA SEMANA FOI Guilherme Piffer COM 33 CAGADAS 🥇
📅 O dia com mais cagadas foi 2024-07-10 com 77 cagadas
🕑 O horário mais ocupado da semana é às 14 com 77 cagadas
🌅 Manhã (6h-12h): 74 cagadas
🌄 Tarde (12h-8h): 77 cagadas
🌃 Noite (18h-24h): 73 cagadas
🌉 Madrugada (0h-6h): 22 cagadas
O período preferido do dia pra cagar é de Tarde com 77 cagadas


In [138]:
print_day_winners(df_chat)

📅🥇 OS MAIORES CAGÕES DO DIA 04/07 FORAM Antonio Seixas e Thiago Bittencourt COM 1 CAGADAS 💩

📅🥇 OS MAIORES CAGÕES DO DIA 05/07 FORAM Leonardo Torres e Laura Debastiani COM 3 CAGADAS 💩

📅🥇 O MAIOR CAGÃO DO DIA 06/07 FOI Antonio Seixas COM 2 CAGADAS 💩

📅🥇 O MAIOR CAGÃO DO DIA 07/07 FOI Duda COM 3 CAGADAS 💩

📅🥇 O MAIOR CAGÃO DO DIA 08/07 FOI Guilherme Piffer COM 3 CAGADAS 💩

📅🥇 O MAIOR CAGÃO DO DIA 09/07 FOI Duda COM 4 CAGADAS 💩

📅🥇 OS MAIORES CAGÕES DO DIA 10/07 FORAM Guilherme Piffer e Leonardo Torres COM 4 CAGADAS 💩

📅🥇 O MAIOR CAGÃO DO DIA 11/07 FOI Laura Debastiani COM 4 CAGADAS 💩

📅🥇 OS MAIORES CAGÕES DO DIA 12/07 FORAM Henrique Lefundes e Rafael COM 4 CAGADAS 💩

📅🥇 OS MAIORES CAGÕES DO DIA 13/07 FORAM Guilherme Piffer, Thiago Bittencourt, Antonio Seixas e Daniel Piazzi COM 2 CAGADAS 💩

📅🥇 O MAIOR CAGÃO DO DIA 14/07 FOI Duda COM 4 CAGADAS 💩

📅🥇 OS MAIORES CAGÕES DO DIA 15/07 FORAM Guilherme Piffer, Thiago Bittencourt, Alice Martins e Leonardo Torres COM 3 CAGADAS 💩

📅🥇 O MAIOR CAGÃO

In [139]:
print_ranking(df_ranking)

🏆 RANKING SEMANAL 🏆
🥇 1. Guilherme Piffer: 33 cagadas
🥈 2. Thiago Bittencourt: 31 cagadas
🥉 3. Antonio Seixas: 26 cagadas
🪠 4. Alice Martins: 26 cagadas
🪠 5. Leonardo Torres: 25 cagadas
🪠 6. Daniel Piazzi: 25 cagadas
🪠 7. Duda: 23 cagadas
🪠 8. Laura Debastiani: 22 cagadas
🫃 9. Henrique Lefundes: 21 cagadas (ACTIVIA ZONE)
🫃 10. Rafael: 19 cagadas (ACTIVIA ZONE)
🫃 11. Duane: 17 cagadas (ACTIVIA ZONE)


In [140]:
for sender in df_chat['Sender'].unique():
    if sender == GROUP_NAME:
        continue
    print_personal_resume(df_chat, df_ranking, sender)

🧻🪠 RESUMO PESSOAL @Antonio Seixas 🪠🧻
⏳ Maior intervalo sem cagar: 1 dia 1 hora 37 minutos
⌛ Menor intervalo sem cagar: 1 hora 44 minutos
🚽 Caga em média 1.73 vezes por dia
🧻 Está na posição 3 do ranking com 26 cagadas

🧻🪠 RESUMO PESSOAL @Guilherme Piffer 🪠🧻
⏳ Maior intervalo sem cagar: 1 dia 6 horas 43 minutos
⌛ Menor intervalo sem cagar: 24 minutos
🚽 Caga em média 2.31 vezes por dia
🧻 Está na posição 1 do ranking com 33 cagadas

🧻🪠 RESUMO PESSOAL @Alice Martins 🪠🧻
⏳ Maior intervalo sem cagar: 2 dias 1 hora 23 minutos
⌛ Menor intervalo sem cagar: 4 horas 41 minutos
🚽 Caga em média 1.83 vezes por dia
🧻 Está na posição 4 do ranking com 26 cagadas

🧻🪠 RESUMO PESSOAL @Daniel Piazzi 🪠🧻
⏳ Maior intervalo sem cagar: 2 dias 6 horas 9 minutos
⌛ Menor intervalo sem cagar: 5 horas 8 minutos
🚽 Caga em média 1.69 vezes por dia
🧻 Está na posição 6 do ranking com 25 cagadas

🧻🪠 RESUMO PESSOAL @Henrique Lefundes 🪠🧻
⏳ Maior intervalo sem cagar: 1 dia 21 horas 12 minutos
⌛ Menor intervalo sem cagar:
🚽 C